In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"] = "TRUE"

import torch

In [2]:
import torch.nn.functional as F
import matplotlib.pyplot as plt # for making figures
%matplotlib inline

In [3]:
words = open('names.txt', 'r').read().splitlines()
words[:8]

['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']

In [4]:
len(words)

32033

In [5]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
print(itos)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}


In [6]:
block_size = 3 # context length: how many characters do we take to predict the next one?
X, Y = [], []
for w in words:
  
  #print(w)
  context = [0] * block_size
  for ch in w + '.':
    ix = stoi[ch]
    X.append(context)
    Y.append(ix)
    #print(''.join(itos[i] for i in context), '--->', itos[ix])
    context = context[1:] + [ix] # crop and append
  
X = torch.tensor(X)
Y = torch.tensor(Y)

In [7]:
X.shape, X.dtype, Y.shape, Y.dtype

(torch.Size([228146, 3]), torch.int64, torch.Size([228146]), torch.int64)

In [8]:
C = torch.randn((27, 2)) 

In [9]:
C[5]

tensor([1.9625, 0.9386])

In [10]:
emb = C[X]

In [11]:
W1 = torch.randn((6,100))
b1 = torch.randn(100)

In [12]:
h = emb.view(emb.shape[0],6) @ W1 + b1 

In [13]:
torch.cat([emb[:, 0, :], emb[:, 1, :], emb[:, 2, :]], 1).shape

torch.Size([228146, 6])

In [14]:
torch.cat(torch.unbind(emb,1))

tensor([[ 1.7091, -0.4200],
        [ 1.7091, -0.4200],
        [ 1.7091, -0.4200],
        ...,
        [ 1.2628, -2.2240],
        [-0.7337, -0.3346],
        [ 0.1008, -1.0809]])

In [15]:
a = torch.arange(18)

In [16]:
a.view(9,2)

tensor([[ 0,  1],
        [ 2,  3],
        [ 4,  5],
        [ 6,  7],
        [ 8,  9],
        [10, 11],
        [12, 13],
        [14, 15],
        [16, 17]])

In [17]:
a.view(3,3,2)

tensor([[[ 0,  1],
         [ 2,  3],
         [ 4,  5]],

        [[ 6,  7],
         [ 8,  9],
         [10, 11]],

        [[12, 13],
         [14, 15],
         [16, 17]]])

In [18]:
h = torch.tanh(emb.view(emb.shape[0],6) @ W1 + b1 )

In [19]:
W2 = torch.randn((100,27))
b2 = torch.randn(27)

In [20]:
logits = h @ W2 + b2

In [21]:
logits.shape

torch.Size([228146, 27])

In [22]:
counts = logits.exp()

In [23]:
prob = counts / counts.sum(1,keepdims=True)

In [32]:
loss = -prob[torch.arange(32), Y[:32]].log().mean()
loss

tensor(14.9953)

In [33]:
X.shape, Y.shape

(torch.Size([228146, 3]), torch.Size([228146]))

In [61]:

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C = torch.randn((27, 2), generator=g)
W1 = torch.randn((6, 100), generator=g)
b1 = torch.randn(100, generator=g)
W2 = torch.randn((100, 27), generator=g)
b2 = torch.randn(27, generator=g)
parameters = [C, W1, b1, W2, b2]

In [62]:
for p in parameters:
    p.requires_grad = True

In [ ]:
for i in range(1000):
    emb = C[X]
    h = torch.tanh(emb.view(-1, 6) @ W1 + b1)
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, Y)
    for p in parameters:
        p.grad = None
    loss.backward()

    for p in parameters:
        p.data += -0.1 * p.grad

print(loss.item())

RuntimeError: shape '[-1, 30]' is invalid for input of size 1368876

In [53]:
sum((p.nelement()) for p in parameters)

11897

In [ ]:
F.cross_entropy(logits, Y)